In [1]:
import geopandas as gpd
from shapely.geometry import Point
from shapely.ops import nearest_points
import pandas as pd

In [2]:
train_gdf  = gpd.read_file(r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\PhD\Mapping\ABC\ABC-model-18-02-2024\results\n_01\inputs\n_1_train.shp")
metro_gdf = gpd.read_file(r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\PhD\Mapping\ABC\ABC-model-18-02-2024\results\n_01\inputs\n_1_metro.shp")
tram_gdf = gpd.read_file(r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\PhD\Mapping\ABC\ABC-model-18-02-2024\results\n_01\inputs\n_1_tram.shp")
bus_gdf = gpd.read_file(r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\PhD\Mapping\ABC\ABC-model-18-02-2024\results\n_01\inputs\n_1_bus.shp")

## Train

### N3_1 Train

In [ ]:
# train
base_path = 'D:\\15_Polimi\\Polimi_onedrive\\OneDrive - Politecnico di Milano\\Lombardia GTFS\\TrenitaliaTrenord\\'

# Load the stop_times file
stop_times_df = pd.read_csv(base_path + 'stop_times.txt')
routes_df = pd.read_csv(base_path + 'routes.txt')
trips_df = pd.read_csv(base_path + 'trips.txt')
stop_times_df = pd.read_csv(base_path + 'stop_times.txt')
stops_df = pd.read_csv (base_path + 'stops.txt')
# Count the number of arrivals for each stop_id across all days
all_stop_frequencies = stop_times_df['stop_id'].value_counts().reset_index()
all_stop_frequencies.columns = ['stop_id', 'N3_1']

# Save the frequency of arrivals for each stop_id across all days to a CSV file
#all_stop_frequencies.to_csv(base_path + 'all_stop_frequencies.csv', index=False)

In [ ]:
train_gdf_N3_1 = train_gdf.merge(all_stop_frequencies, on='stop_id', how='left')
train_gdf_N3_1

### N3_2

In [ ]:
# Join trips with routes to get route_id for each trip
trips_with_routes = trips_df.merge(routes_df, on='route_id', how='left')

# Join stop_times with trips to get route information for each stop time
stop_times_with_routes = stop_times_df.merge(trips_with_routes[['trip_id', 'route_id']], on='trip_id', how='left')

# Count unique routes for each stop_id
routes_per_stop = stop_times_with_routes.groupby('stop_id')['route_id'].nunique().reset_index()
routes_per_stop.columns = ['stop_id', 'N3_2']

# Save the unique routes per stop to a CSV file
#routes_per_stop.to_csv(base_path + 'routes_per_stop.csv', index=False)

In [ ]:
routes_per_stop

In [ ]:
train_gdf_N3_2 = train_gdf_N3_1.merge(routes_per_stop, on='stop_id', how='left')
train_gdf_N3_2

### N3_3

In [ ]:
route_trip = pd.merge(routes_df, trips_df, on='route_id')
# Remove duplicates by route_id and keep the first occurrence
df_unique_routes = route_trip.drop_duplicates(subset='route_id', keep='first')
trip_stop_times = pd.merge(df_unique_routes, stop_times_df, on='trip_id')
trip_stop_times_sorted = trip_stop_times.sort_values(by=['route_id', 'stop_sequence'])
trip_stop_times_sorted.head(20)
m=pd.merge(trip_stop_times_sorted,stops_df,on='stop_id')
m=m.sort_values(by=['route_id', 'stop_sequence'])
def stops_to_wkt(group):
    # Sort by 'stop_sequence'
    sorted_group = group.sort_values(by='stop_sequence')
    # Create the WKT MULTILINESTRING with the sorted coordinates
    wkt_line = 'MULTILINESTRING((' + ', '.join(
        [f"{lon} {lat}" for lat, lon in zip(sorted_group['stop_lat'], sorted_group['stop_lon'])]
    ) + '))'
    return wkt_line

# Apply the function to each group
n = m.groupby('route_id').apply(stops_to_wkt).reset_index(name='geometry')
df=pd.merge(n,routes_df,on='route_id')
df.to_csv(r"C:\Users\mafgo\Downloads\New folder (2)\xzy.csv")

In [ ]:
#load_route_lebgth
train_length= pd.read_csv(r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\PhD\Mapping\ABC\ABC-model-18-02-2024\results\n_03\inputs\train_length.csv")

In [ ]:
train_length

In [ ]:
# Load GTFS stops data
stops = stops_df
stop_times = stop_times_df
routes = routes_df
trips = trips_df

In [ ]:
# Filter the stops data to only include the stop_ids from train_gdf
filtered_stops = stops[stops['stop_id'].isin(train_gdf['stop_id'])]

# Filter the stop times data to only include the relevant stop_ids
filtered_stop_times = stop_times[stop_times['stop_id'].isin(filtered_stops['stop_id'])]

# Merge stop times with trips to get route_id for each stop time
filtered_stop_times = filtered_stop_times.merge(trips[['trip_id', 'route_id']], on='trip_id')

# Merge the above with filtered stops to get the stop names
filtered_stop_times = filtered_stop_times.merge(filtered_stops[['stop_id', 'stop_name']], on='stop_id')

# Merge with routes to get the route names
filtered_stop_times = filtered_stop_times.merge(routes[['route_id', 'route_long_name']], on='route_id')

# Assuming you have loaded your train lines length data into a DataFrame called train_length
# Merge this with your train_length DataFrame to get the lengths of routes
stop_routes = filtered_stop_times.merge(train_length[['route_id', 'len_km']], on='route_id')

# Group by stop_name and find the longest route for each stop
longest_routes_per_stop = stop_routes.groupby('stop_name')['len_km'].max().reset_index()

# Merge to get the route names of the longest routes
longest_routes_per_stop = longest_routes_per_stop.merge(stop_routes[['stop_name', 'route_long_name', 'len_km']], on=['stop_name', 'len_km'])

# Now longest_routes_per_stop contains the stop names and the longest route that passes through each stop
print(longest_routes_per_stop)

In [ ]:
longest_routes_per_stop = stop_routes.groupby('stop_name').apply(
    lambda g: g.loc[g['len_km'].idxmax()]
).reset_index(drop=True)

# Select the distinct columns including stop_id
longest_routes_per_stop = longest_routes_per_stop[['stop_id', 'stop_name', 'route_long_name', 'len_km']].drop_duplicates()

# Now longest_routes_per_stop contains the stop names and the longest route that passes through each stop, without duplicates
print(longest_routes_per_stop)

In [ ]:
longest_routes_per_stop

In [ ]:
Train = longest_routes_per_stop.rename(columns={'len_km': 'N3_3'})
Train

In [ ]:
combined_df = train_gdf_N3_2.merge(Train, on='stop_id', how='left')
combined_df.to_csv(r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\PhD\Mapping\ABC\ABC-model-18-02-2024\results\n_03\outputs\train\train.csv")

## Metro

### N3_1 Metro

In [ ]:
# train
base_path = r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\Lombardia GTFS\ATM-UTF8"

# Load the stop_times file
stop_times_df = pd.read_csv(r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\Lombardia GTFS\ATM-UTF8\stop_times.txt")
routes_df = pd.read_csv(r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\Lombardia GTFS\ATM-UTF8\routes.txt")
trips_df = pd.read_csv(r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\Lombardia GTFS\ATM-UTF8\trips.txt")
stop_times_df = pd.read_csv(r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\Lombardia GTFS\ATM-UTF8\stop_times.txt")
stops_df = pd.read_csv (r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\Lombardia GTFS\ATM-UTF8\stops.txt")
# Count the number of arrivals for each stop_id across all days
all_stop_frequencies = stop_times_df['stop_id'].value_counts().reset_index()
all_stop_frequencies.columns = ['stop_id', 'N3_1']

# Save the frequency of arrivals for each stop_id across all days to a CSV file
#all_stop_frequencies.to_csv(base_path + 'all_stop_frequencies.csv', index=False)

In [ ]:
metro_gdf_N3_1 = metro_gdf.merge(all_stop_frequencies, on='stop_id', how='left')
metro_gdf_N3_1

### N3_2 Metro

In [ ]:
# Join trips with routes to get route_id for each trip
trips_with_routes = trips_df.merge(routes_df, on='route_id', how='left')

# Join stop_times with trips to get route information for each stop time
stop_times_with_routes = stop_times_df.merge(trips_with_routes[['trip_id', 'route_id']], on='trip_id', how='left')

# Count unique routes for each stop_id
routes_per_stop = stop_times_with_routes.groupby('stop_id')['route_id'].nunique().reset_index()
routes_per_stop.columns = ['stop_id', 'N3_2']

# Save the unique routes per stop to a CSV file
#routes_per_stop.to_csv(base_path + 'routes_per_stop.csv', index=False)

In [ ]:
routes_per_stop

In [ ]:
metro_gdf_N3_2 = metro_gdf_N3_1.merge(routes_per_stop, on='stop_id', how='left')
metro_gdf_N3_2

### N3_3 Metro

In [ ]:
route_trip = pd.merge(routes_df, trips_df, on='route_id')
# Remove duplicates by route_id and keep the first occurrence
df_unique_routes = route_trip.drop_duplicates(subset='route_id', keep='first')
trip_stop_times = pd.merge(df_unique_routes, stop_times_df, on='trip_id')
trip_stop_times_sorted = trip_stop_times.sort_values(by=['route_id', 'stop_sequence'])
trip_stop_times_sorted.head(20)
m=pd.merge(trip_stop_times_sorted,stops_df,on='stop_id')
m=m.sort_values(by=['route_id', 'stop_sequence'])
def stops_to_wkt(group):
    # Sort by 'stop_sequence'
    sorted_group = group.sort_values(by='stop_sequence')
    # Create the WKT MULTILINESTRING with the sorted coordinates
    wkt_line = 'MULTILINESTRING((' + ', '.join(
        [f"{lon} {lat}" for lat, lon in zip(sorted_group['stop_lat'], sorted_group['stop_lon'])]
    ) + '))'
    return wkt_line

# Apply the function to each group
n = m.groupby('route_id').apply(stops_to_wkt).reset_index(name='geometry')
df=pd.merge(n,routes_df,on='route_id')
#df.to_csv(r"C:\Users\mafgo\Downloads\New folder (2)\xzy.csv")

In [ ]:
metro_df=df[df['route_type'] == 1]

In [ ]:
metro_df.to_csv(r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\PhD\Mapping\ABC\ABC-model-18-02-2024\results\n_03\inputs\metro.csv")

In [ ]:
#load_route_lebgth
metro_length= pd.read_csv(r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\PhD\Mapping\ABC\ABC-model-18-02-2024\results\n_03\inputs\metro_length.csv")

In [ ]:
metro_length

In [ ]:
# Load GTFS stops data
stops = stops_df
stop_times = stop_times_df
routes = routes_df
trips = trips_df

In [ ]:
# Filter the stops data to only include the stop_ids from train_gdf
filtered_stops = stops[stops['stop_id'].isin(metro_gdf['stop_id'])]

# Filter the stop times data to only include the relevant stop_ids
filtered_stop_times = stop_times[stop_times['stop_id'].isin(filtered_stops['stop_id'])]

# Merge stop times with trips to get route_id for each stop time
filtered_stop_times = filtered_stop_times.merge(trips[['trip_id', 'route_id']], on='trip_id')

# Merge the above with filtered stops to get the stop names
filtered_stop_times = filtered_stop_times.merge(filtered_stops[['stop_id', 'stop_name']], on='stop_id')

# Merge with routes to get the route names
filtered_stop_times = filtered_stop_times.merge(routes[['route_id', 'route_long_name']], on='route_id')

# Assuming you have loaded your train lines length data into a DataFrame called train_length
# Merge this with your train_length DataFrame to get the lengths of routes
stop_routes = filtered_stop_times.merge(metro_length[['route_id', 'len_km']], on='route_id')

# Group by stop_name and find the longest route for each stop
longest_routes_per_stop = stop_routes.groupby('stop_name')['len_km'].max().reset_index()

# Merge to get the route names of the longest routes
longest_routes_per_stop = longest_routes_per_stop.merge(stop_routes[['stop_name', 'route_long_name', 'len_km']], on=['stop_name', 'len_km'])

# Now longest_routes_per_stop contains the stop names and the longest route that passes through each stop
print(longest_routes_per_stop)

In [ ]:
longest_routes_per_stop = stop_routes.groupby('stop_name').apply(
    lambda g: g.loc[g['len_km'].idxmax()]
).reset_index(drop=True)

# Select the distinct columns including stop_id
longest_routes_per_stop = longest_routes_per_stop[['stop_id', 'stop_name', 'route_long_name', 'len_km']].drop_duplicates()

# Now longest_routes_per_stop contains the stop names and the longest route that passes through each stop, without duplicates
print(longest_routes_per_stop)

In [ ]:
Metro = longest_routes_per_stop.rename(columns={'len_km': 'N3_3'})
Metro

In [ ]:
combined_df = metro_gdf_N3_2.merge(Metro, on='stop_id', how='left')
combined_df.to_csv(r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\PhD\Mapping\ABC\ABC-model-18-02-2024\results\n_03\outputs\metro\metro.csv")

## BUS

### N3_1 bus

In [ ]:
# train
base_path = r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\Lombardia GTFS\ATM-UTF8"

# Load the stop_times file
stop_times_df = pd.read_csv(r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\Lombardia GTFS\ATM-UTF8\stop_times.txt")
routes_df = pd.read_csv(r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\Lombardia GTFS\ATM-UTF8\routes.txt")
trips_df = pd.read_csv(r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\Lombardia GTFS\ATM-UTF8\trips.txt")
stop_times_df = pd.read_csv(r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\Lombardia GTFS\ATM-UTF8\stop_times.txt")
stops_df = pd.read_csv (r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\Lombardia GTFS\ATM-UTF8\stops.txt")
# Count the number of arrivals for each stop_id across all days
all_stop_frequencies = stop_times_df['stop_id'].value_counts().reset_index()
all_stop_frequencies.columns = ['stop_id', 'N3_1']

# Save the frequency of arrivals for each stop_id across all days to a CSV file
#all_stop_frequencies.to_csv(base_path + 'all_stop_frequencies.csv', index=False)

In [ ]:
bus_gdf_N3_1 = bus_gdf.merge(all_stop_frequencies, on='stop_id', how='left')
bus_gdf_N3_1

### N3_2 Bus

In [ ]:
# Join trips with routes to get route_id for each trip
trips_with_routes = trips_df.merge(routes_df, on='route_id', how='left')

# Join stop_times with trips to get route information for each stop time
stop_times_with_routes = stop_times_df.merge(trips_with_routes[['trip_id', 'route_id']], on='trip_id', how='left')

# Count unique routes for each stop_id
routes_per_stop = stop_times_with_routes.groupby('stop_id')['route_id'].nunique().reset_index()
routes_per_stop.columns = ['stop_id', 'N3_2']

# Save the unique routes per stop to a CSV file
#routes_per_stop.to_csv(base_path + 'routes_per_stop.csv', index=False)

In [ ]:
routes_per_stop

In [ ]:
bus_gdf_N3_2 = bus_gdf_N3_1.merge(routes_per_stop, on='stop_id', how='left')
bus_gdf_N3_2

### N3_3 Bus

In [ ]:
route_trip = pd.merge(routes_df, trips_df, on='route_id')
# Remove duplicates by route_id and keep the first occurrence
df_unique_routes = route_trip.drop_duplicates(subset='route_id', keep='first')
trip_stop_times = pd.merge(df_unique_routes, stop_times_df, on='trip_id')
trip_stop_times_sorted = trip_stop_times.sort_values(by=['route_id', 'stop_sequence'])
trip_stop_times_sorted.head(20)
m=pd.merge(trip_stop_times_sorted,stops_df,on='stop_id')
m=m.sort_values(by=['route_id', 'stop_sequence'])
def stops_to_wkt(group):
    # Sort by 'stop_sequence'
    sorted_group = group.sort_values(by='stop_sequence')
    # Create the WKT MULTILINESTRING with the sorted coordinates
    wkt_line = 'MULTILINESTRING((' + ', '.join(
        [f"{lon} {lat}" for lat, lon in zip(sorted_group['stop_lat'], sorted_group['stop_lon'])]
    ) + '))'
    return wkt_line

# Apply the function to each group
n = m.groupby('route_id').apply(stops_to_wkt).reset_index(name='geometry')
df=pd.merge(n,routes_df,on='route_id')
#df.to_csv(r"C:\Users\mafgo\Downloads\New folder (2)\xzy.csv")

In [ ]:
bus_df=df[df['route_type'] == 3]

In [ ]:
bus_df.to_csv(r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\PhD\Mapping\ABC\ABC-model-18-02-2024\results\n_03\inputs\bus.csv")

In [ ]:
bus_df

In [ ]:
#load_route_lebgth
bus_length= pd.read_csv(r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\PhD\Mapping\ABC\ABC-model-18-02-2024\results\n_03\inputs\bus_length.csv")

In [ ]:
# Load GTFS stops data
stops = stops_df
stop_times = stop_times_df
routes = routes_df
trips = trips_df

In [ ]:
# Filter the stops data to only include the stop_ids from train_gdf
filtered_stops = stops[stops['stop_id'].isin(bus_gdf['stop_id'])]

# Filter the stop times data to only include the relevant stop_ids
filtered_stop_times = stop_times[stop_times['stop_id'].isin(filtered_stops['stop_id'])]

# Merge stop times with trips to get route_id for each stop time
filtered_stop_times = filtered_stop_times.merge(trips[['trip_id', 'route_id']], on='trip_id')

# Merge the above with filtered stops to get the stop names
filtered_stop_times = filtered_stop_times.merge(filtered_stops[['stop_id', 'stop_name']], on='stop_id')

# Merge with routes to get the route names
filtered_stop_times = filtered_stop_times.merge(routes[['route_id', 'route_long_name']], on='route_id')

# Assuming you have loaded your train lines length data into a DataFrame called train_length
# Merge this with your train_length DataFrame to get the lengths of routes
stop_routes = filtered_stop_times.merge(metro_length[['route_id', 'len_km']], on='route_id')

# Group by stop_name and find the longest route for each stop
longest_routes_per_stop = stop_routes.groupby('stop_name')['len_km'].max().reset_index()

# Merge to get the route names of the longest routes
longest_routes_per_stop = longest_routes_per_stop.merge(stop_routes[['stop_name', 'route_long_name', 'len_km']], on=['stop_name', 'len_km'])

# Now longest_routes_per_stop contains the stop names and the longest route that passes through each stop
print(longest_routes_per_stop)

In [ ]:
longest_routes_per_stop = stop_routes.groupby('stop_name').apply(
    lambda g: g.loc[g['len_km'].idxmax()]
).reset_index(drop=True)

# Select the distinct columns including stop_id
longest_routes_per_stop = longest_routes_per_stop[['stop_id', 'stop_name', 'route_long_name', 'len_km']].drop_duplicates()

# Now longest_routes_per_stop contains the stop names and the longest route that passes through each stop, without duplicates
print(longest_routes_per_stop)

In [ ]:
Bus = longest_routes_per_stop.rename(columns={'len_km': 'N3_3'})
Bus

In [ ]:
combined_df = bus_gdf_N3_2.merge(Bus, on='stop_id', how='left')
combined_df.to_csv(r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\PhD\Mapping\ABC\ABC-model-18-02-2024\results\n_03\outputs\n03-bus.csv")

## TRAM

## N3_1 Tram

In [3]:
# train
base_path = r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\Lombardia GTFS\ATM-UTF8"

# Load the stop_times file
stop_times_df = pd.read_csv(r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\Lombardia GTFS\ATM-UTF8\stop_times.txt")
routes_df = pd.read_csv(r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\Lombardia GTFS\ATM-UTF8\routes.txt")
trips_df = pd.read_csv(r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\Lombardia GTFS\ATM-UTF8\trips.txt")
stop_times_df = pd.read_csv(r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\Lombardia GTFS\ATM-UTF8\stop_times.txt")
stops_df = pd.read_csv (r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\Lombardia GTFS\ATM-UTF8\stops.txt")
# Count the number of arrivals for each stop_id across all days
all_stop_frequencies = stop_times_df['stop_id'].value_counts().reset_index()
all_stop_frequencies.columns = ['stop_id', 'N3_1']

# Save the frequency of arrivals for each stop_id across all days to a CSV file
#all_stop_frequencies.to_csv(base_path + 'all_stop_frequencies.csv', index=False)

In [4]:
tram_gdf_N3_1 = tram_gdf.merge(all_stop_frequencies, on='stop_id', how='left')

## N3_2 Tram


In [5]:
# Join trips with routes to get route_id for each trip
trips_with_routes = trips_df.merge(routes_df, on='route_id', how='left')

# Join stop_times with trips to get route information for each stop time
stop_times_with_routes = stop_times_df.merge(trips_with_routes[['trip_id', 'route_id']], on='trip_id', how='left')

# Count unique routes for each stop_id
routes_per_stop = stop_times_with_routes.groupby('stop_id')['route_id'].nunique().reset_index()
routes_per_stop.columns = ['stop_id', 'N3_2']

# Save the unique routes per stop to a CSV file
#routes_per_stop.to_csv(base_path + 'routes_per_stop.csv', index=False)

In [6]:
routes_per_stop

,stop_id,N3_2
0,14826,1
1,14827,1
2,14828,3
3,14831,1
4,14832,1
...,...,...
5827,73543,2
5828,73544,1
5829,73545,1
5830,73578,1


In [7]:
tram_gdf_N3_2 = tram_gdf_N3_1.merge(routes_per_stop, on='stop_id', how='left')
tram_gdf_N3_2

,field_1,Unnamed_ 0,stop_id,stop_name,stop_desc,pums,s_lat,s_lon,int_scr,scr_abc,Cluster,catch_area,geometry,N3_1,N3_2
0,126.0,0,14827.0,P.ta Venezia M1,P.ta Venezia M1,no,516076.7226,5.035674e+06,4,12,High,700,POINT (516076.723 5035673.715),396.0,1.0
1,117.0,1,14828.0,P.le Principessa Clotilde (Osp. Fatebenefratelli),P.le Principessa Clotilde (Osp. Fatebenefratelli),no,515055.3403,5.036331e+06,4,7,Medium,280,POINT (515055.340 5036330.569),787.0,3.0
2,110.0,2,14838.0,P.le Cim.Monumentale Via Bramante,P.le Cim.Monumentale Via Bramante,no,513887.8555,5.036715e+06,4,7,Medium,280,POINT (513887.856 5036714.603),635.0,2.0
3,268.0,3,14842.0,"V.le Ungheria, 20","V.le Ungheria, 20",no,519258.6787,5.032512e+06,4,7,Medium,280,POINT (519258.679 5032511.627),542.0,4.0
4,46.0,4,14846.0,Cenisio M5,Cenisio M5,no,513519.0908,5.037123e+06,4,9,High,700,POINT (513519.091 5037123.350),644.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
446,NaN,784,NaN,None,None,yes,518982.4610,5.031795e+06,4,5,Low,175,POINT (518982.461 5031794.983),NaN,NaN
447,NaN,785,NaN,None,None,yes,519447.2682,5.031958e+06,4,4,Low,175,POINT (519447.268 5031958.369),NaN,NaN
448,NaN,786,NaN,None,None,yes,519558.5402,5.032082e+06,4,4,Low,175,POINT (519558.540 5032082.318),NaN,NaN
449,NaN,787,NaN,None,None,yes,519916.3008,5.032422e+06,4,4,Low,175,POINT (519916.301 5032421.768),NaN,NaN


## TRAM N3_3

In [8]:
route_trip = pd.merge(routes_df, trips_df, on='route_id')
# Remove duplicates by route_id and keep the first occurrence
df_unique_routes = route_trip.drop_duplicates(subset='route_id', keep='first')
trip_stop_times = pd.merge(df_unique_routes, stop_times_df, on='trip_id')
trip_stop_times_sorted = trip_stop_times.sort_values(by=['route_id', 'stop_sequence'])
trip_stop_times_sorted.head(20)
m=pd.merge(trip_stop_times_sorted,stops_df,on='stop_id')
m=m.sort_values(by=['route_id', 'stop_sequence'])
def stops_to_wkt(group):
    # Sort by 'stop_sequence'
    sorted_group = group.sort_values(by='stop_sequence')
    # Create the WKT MULTILINESTRING with the sorted coordinates
    wkt_line = 'MULTILINESTRING((' + ', '.join(
        [f"{lon} {lat}" for lat, lon in zip(sorted_group['stop_lat'], sorted_group['stop_lon'])]
    ) + '))'
    return wkt_line

# Apply the function to each group
n = m.groupby('route_id').apply(stops_to_wkt).reset_index(name='geometry')
df=pd.merge(n,routes_df,on='route_id')
#df.to_csv(r"C:\Users\mafgo\Downloads\New folder (2)\xzy.csv")

In [9]:
tram_df=df[df['route_type'] == 0]

In [10]:
tram_df.to_csv(r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\PhD\Mapping\ABC\ABC-model-18-02-2024\results\n_03\inputs\tram.csv")

In [11]:
#load_route_lebgth
tram_length= pd.read_csv(r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\PhD\Mapping\ABC\ABC-model-18-02-2024\results\n_03\inputs\tram_length.csv")

In [12]:
tram_length

,field_1,route_id,route_shor,route_long,agency_id,route_type,len_km
0,0,1600,1 A,Greco - Roserio,47,0,13.21468
1,2,1602,2 A,P.le. Bausan - P.le Negrelli,47,0,10.11565
2,3,1603,1 R,Roserio - Greco,47,0,13.24758
3,4,1604,2 R,P.le Negrelli - P.le. Bausan,47,0,10.08507
4,5,1605,3 A,Gratosoglio - Duomo,47,0,7.04070
5,6,1606,3 R,Duomo - Gratosoglio,47,0,7.03180
6,7,1607,4 A,Niguarda (Parco Nord) - Maciachini,47,0,6.53673
7,8,1608,4 R,Maciachini - Niguarda (Parco Nord),47,0,1.52179
8,9,1609,5 A,Ortica - Ospedale Maggiore,47,0,9.79254
9,10,1610,5 R,Ospedale Maggiore - Ortica,47,0,9.74291


In [13]:
# Load GTFS stops data
stops = stops_df
stop_times = stop_times_df
routes = routes_df
trips = trips_df

In [14]:
# Filter the stops data to only include the stop_ids from train_gdf
filtered_stops = stops[stops['stop_id'].isin(tram_gdf['stop_id'])]

# Filter the stop times data to only include the relevant stop_ids
filtered_stop_times = stop_times[stop_times['stop_id'].isin(filtered_stops['stop_id'])]

# Merge stop times with trips to get route_id for each stop time
filtered_stop_times = filtered_stop_times.merge(trips[['trip_id', 'route_id']], on='trip_id')

# Merge the above with filtered stops to get the stop names
filtered_stop_times = filtered_stop_times.merge(filtered_stops[['stop_id', 'stop_name']], on='stop_id')

# Merge with routes to get the route names
filtered_stop_times = filtered_stop_times.merge(routes[['route_id', 'route_long_name']], on='route_id')

# Assuming you have loaded your train lines length data into a DataFrame called train_length
# Merge this with your train_length DataFrame to get the lengths of routes
stop_routes = filtered_stop_times.merge(tram_length[['route_id', 'len_km']], on='route_id')

# Group by stop_name and find the longest route for each stop
longest_routes_per_stop = stop_routes.groupby('stop_name')['len_km'].max().reset_index()

# Merge to get the route names of the longest routes
longest_routes_per_stop = longest_routes_per_stop.merge(stop_routes[['stop_name', 'route_long_name', 'len_km']], on=['stop_name', 'len_km'])

# Now longest_routes_per_stop contains the stop names and the longest route that passes through each stop
print(longest_routes_per_stop)

               stop_name    len_km                       route_long_name
0       Abbiategrasso M2  12.93822  Duomo M1 M3 - Rozzano (Via G. Rossa)
1       Abbiategrasso M2  12.93822  Duomo M1 M3 - Rozzano (Via G. Rossa)
2       Abbiategrasso M2  12.93822  Duomo M1 M3 - Rozzano (Via G. Rossa)
3       Abbiategrasso M2  12.93822  Duomo M1 M3 - Rozzano (Via G. Rossa)
4       Abbiategrasso M2  12.93822  Duomo M1 M3 - Rozzano (Via G. Rossa)
...                  ...       ...                                   ...
113236        Zara M3/M5   9.74291            Ospedale Maggiore - Ortica
113237        Zara M3/M5   9.74291            Ospedale Maggiore - Ortica
113238        Zara M3/M5   9.74291            Ospedale Maggiore - Ortica
113239        Zara M3/M5   9.74291            Ospedale Maggiore - Ortica
113240        Zara M3/M5   9.74291            Ospedale Maggiore - Ortica

[113241 rows x 3 columns]


In [15]:
longest_routes_per_stop = stop_routes.groupby('stop_name').apply(
    lambda g: g.loc[g['len_km'].idxmax()]
).reset_index(drop=True)

# Select the distinct columns including stop_id
longest_routes_per_stop = longest_routes_per_stop[['stop_id', 'stop_name', 'route_long_name', 'len_km']].drop_duplicates()

# Now longest_routes_per_stop contains the stop names and the longest route that passes through each stop, without duplicates
print(longest_routes_per_stop)

     stop_id                       stop_name  \
0      19523                Abbiategrasso M2   
1      18796                         Agrippa   
2      47392          Albertinelli - Segesta   
3      17505                      Anassagora   
4      17103         Arcimboldi Ateneo Nuovo   
..       ...                             ...   
360    18026    Via del Turchino Via Maspero   
361    16893                       Vigentino   
362    16346  Villa Ghirlanda (Cinisello B.)   
363    17118                        Volvinio   
364    16042                      Zara M3/M5   

                          route_long_name    len_km  
0    Duomo M1 M3 - Rozzano (Via G. Rossa)  12.93822  
1    Rozzano (Via G. Rossa) - Duomo M1 M3  12.91781  
2       Monte Velino - San Siro Stadio M5   9.80010  
3                 P.le Lagosta - Precotto   5.43457  
4                 P.le Lagosta - Precotto   5.43457  
..                                    ...       ...  
360     San Siro Stadio M5 - Monte Velino   9

In [16]:
Tram = longest_routes_per_stop.rename(columns={'len_km': 'N3_3'})
Tram

,stop_id,stop_name,route_long_name,N3_3
0,19523,Abbiategrasso M2,Duomo M1 M3 - Rozzano (Via G. Rossa),12.93822
1,18796,Agrippa,Rozzano (Via G. Rossa) - Duomo M1 M3,12.91781
2,47392,Albertinelli - Segesta,Monte Velino - San Siro Stadio M5,9.80010
3,17505,Anassagora,P.le Lagosta - Precotto,5.43457
4,17103,Arcimboldi Ateneo Nuovo,P.le Lagosta - Precotto,5.43457
...,...,...,...,...
360,18026,Via del Turchino Via Maspero,San Siro Stadio M5 - Monte Velino,9.81734
361,16893,Vigentino,Vigentino - P.za Fontana,8.36976
362,16346,Villa Ghirlanda (Cinisello B.),Bicocca M5 - Cinisello B.,5.42473
363,17118,Volvinio,Rozzano (Via G. Rossa) - Duomo M1 M3,12.91781


In [17]:
combined_df = tram_gdf_N3_2.merge(Tram, on='stop_id', how='left')
combined_df.to_csv(r"D:\15_Polimi\Polimi_onedrive\OneDrive - Politecnico di Milano\PhD\Mapping\ABC\ABC-model-18-02-2024\results\n_03\outputs\tram\tram.csv")